In [56]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [57]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

In [58]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'

def get_page_text(url, user_agent=USER_AGENT, error_log=True):
    headers = {'User-Agent': user_agent}
    response = requests.get(url, headers=headers, allow_redirects=True)

    if response.status_code == 200:
        page_text = response.text
        return page_text
    elif error_log:
        print(f'Ошибка! Response code: {response.status_code}')
        print(f'URL: {url}')
        print('--------------')
    
    return None

In [59]:
index = 1
url = f'https://em-russia.ru/shop/all/?PAGEN_1={index}&SIZEN_1=16'

page_text = get_page_text(url, USER_AGENT)

In [60]:
url

'https://em-russia.ru/shop/all/?PAGEN_1=1&SIZEN_1=16'

In [61]:
print(page_text[:1000])

<!doctype html>
<html>
<head>
    <!-- Yandex.Metrika counter -->
<script type="text/javascript" >
   (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};
   m[i].l=1*new Date();
   for (var j = 0; j < document.scripts.length; j++) {if (document.scripts[j].src === r) { return; }}
   k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})
   (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");

   ym(96850838, "init", {
        clickmap:true,
        trackLinks:true,
        accurateTrackBounce:true,
        webvisor:true
   });
</script>
<noscript><div><img src="https://mc.yandex.ru/watch/96850838" style="position:absolute; left:-9999px;" alt="" /></div></noscript>
<!-- /Yandex.Metrika counter -->

    <meta property="og:type" content="website"/>
    <meta property="og:site_name" content="em-russia.ru"/>
    <meta property="og:title" content=""/>
    <meta property="og:url" content="htt

In [62]:
products = pd.DataFrame(columns=['title', 'url', 'price', 'text'])

breakpoint = 0

for page in (1,2,3):

    print('dowload page:', page)
    url = f'https://em-russia.ru/shop/all/?PAGEN_1={page}&SIZEN_1=16'

    page_text = get_page_text(url, USER_AGENT)

    soup = BeautifulSoup(page_text)
    posts_list = soup.find_all('div', {'class': 'product__title'})   
    for p in posts_list:
        
        lnk = p.find('a').attrs['href']
        title = re.sub('[^А-ЯЁа-яё0-9\s]', ' ', p.text)
        title = re.sub('\s\s+', ' ', title)
        
        page_url = 'https://em-russia.ru' + lnk
        clean_path = '/'.join([d for d in page_url.split('/')[2:] if len(d) > 0])
        
        page_text = get_page_text(page_url, USER_AGENT)
        
        if page_text is None:
            continue
        
        text = title + '\n'
        product_page = BeautifulSoup(page_text)
        main_div = product_page.find('div', {'id': 'main'})
        
        cd = main_div.find('div', {'class': 'container'}, recursive=False) 
        
        desc = cd.find('div')
        
        if desc is not None:
            text = text + desc.text.strip()       
        else:
            print(desc)

        tabs = cd.find_all('div', {'class': 'product-nav-tabs__button'})
        
        if tabs is None:
            continue
        else:
            tabs_content = cd.find_all('div', {'class': 'product-tab'})
            for t, tc in zip(tabs, tabs_content):
                text = text + '\n' + t.text.strip() + '\r' +tc.text.strip()
        
        
        text = re.sub('[\n\r]+', '\n', text)
        text = re.sub('[\t]+', ' ', text)
        
        price = main_div.find('span', {'id': 'item-price'})
        price = re.sub('[^0-9]', '', price.text.strip())
        
        
        products.loc[ len(products.index )] = [title, page_url, price, text]
            
        #breakpoint = breakpoint+1
        #if breakpoint>5:
        #    break 
            



dowload page: 1
dowload page: 2
dowload page: 3


In [63]:
print(products.iloc[2]['text'])

ЭМ БИО концентрат 1 ВОСТОК ЭМ 1 
Набор для самостоятельного активирования (приготовления) 3-х литров ЭМ-БИО («ЕМ·1 микробиологическое удобрение «Восток ЭМ-1») микробиологического удобрения для всех видов садовых и комнатных растений.
  Концентрат ЭМ-БИО (ЕМ·1 микробиологическое удобрение «Восток ЭМ-1») представляет собой концентрированную культуру эффективных микроорганизмов, находящуюся в устойчивом неактивном состоянии и предназначенную для длительного хранения, визуально это – светло-коричневая жидкость с приятным кисломолочным запахом.
  Срок годности: 12 месяцев с даты изготовления
Состав
В набор входит:
концентрат ЭМ-БИО (ЕМ·1 микробиологическое удобрение «Восток ЭМ-1») в количестве  30 мл (3 ампулы по 10 мл)
патока тростниковая 30 мл (3 ампулы по 10 мл) 
инструкция по разведению и применению. 
  СОСТАВ концентрата: вода питьевая, патока, комплекс молочнокислых бактерий, дрожжей и продуктов их жизнедеятельности. В 1 см3 содержится не менее 1*106 КОЕ молочнокислых бактерий и 1*104

In [64]:
products.to_csv('data/texts.csv', index=False)

In [65]:
products.describe()

,title,url,price,text
count,45,45,45,45
unique,32,45,34,38
top,ЭМ БИО удобрение 1 Восток ЭМ 1,https://em-russia.ru/shop/all/vozrozhdennoe-bu...,5380,Средство для уборки ЭМ Спрей \nУстраняет непри...
freq,5,1,3,4


## Парсинг страниц базы знаний

In [74]:
from os import makedirs
from os import walk
import os

In [68]:
for index in range(1, 5):
    url = f'https://em-russia.ru/base/?PAGEN_1={index}&SIZEN_1=99'
    page_text = get_page_text(url, USER_AGENT)
    
    soup = BeautifulSoup(page_text, 'html.parser')
    links = soup.find_all('a', class_='article-preview__content')
    urls = [link.get('href') for link in links]

    for u in urls:
        page_url = 'https://em-russia.ru' + u
        clean_path = '/'.join([d for d in page_url.split('/')[2:] if len(d) > 0])
        article_text = get_page_text(page_url, USER_AGENT)

        dir_path = 'data/raw_pages/' + '/'.join(clean_path.split('/')[:-1])
        makedirs(dir_path, exist_ok=True)
        with open(dir_path + '/' + clean_path.split('/')[-1] + '.html', 'w', encoding='utf-8') as f:
            f.write(article_text)

In [69]:
pages_list = []

for dirpath, dirnames, filenames in walk('data/raw_pages'):
    if '.ipynb_checkpoints' in dirpath:
        continue
    
    for fn in filenames:
        if '.DS_Store' in fn:
            continue 
            
        fp = f'{dirpath}/{fn}'
        pages_list.append(fp)

In [70]:
pages_list[:10]

['data/raw_pages/em-russia.ru/base/krasnoyarskiy-kray-kartofel-bez-parshi-fitoftoroza-i-alternarioza-30-k-urozhaynosti.html',
 'data/raw_pages/em-russia.ru/base/instruktsiya-kak-izgotovit-em-kolobki.html',
 'data/raw_pages/em-russia.ru/base/ogorod-bez-khimii.html',
 'data/raw_pages/em-russia.ru/base/kompleksnoe-primenenie-em-v-restorane-peru2.html',
 'data/raw_pages/em-russia.ru/base/10-prichin-mulchirovat-pochvu-i-pro-em-ne-zabyt.html',
 'data/raw_pages/em-russia.ru/base/instruktsiya-po-pererabotke-navoza-pomyeta-s-pomoshchyu-preparata-em-vita.html',
 'data/raw_pages/em-russia.ru/base/byt-ekologiya-septik-biotualety-instruktsiya.html',
 'data/raw_pages/em-russia.ru/base/ustranenie-zapakha-pochvogrunta-i-obezvozhennogo-osadka-na-kanalizatsionnykh-ochistnykh-sooruzheniya.html',
 'data/raw_pages/em-russia.ru/base/akvarium-ustali-myt.html',
 'data/raw_pages/em-russia.ru/base/demonstratsionnyy-opyt-effektivnosti-preparata-em-bio-vostok-em-1-v-samarskoy-oblasti.html']

In [71]:
def clean(text):
    text = re.sub('\<[^\<\>]+\>', ' ', text)
    text = re.sub('&nbsp;', ' ', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('\s\s+', ' ', text)
    text = re.sub('^[\+\-] ', '', text)
    text = diff = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF]', ' ', text)
    text = re.sub(r'[^\x00-\x7F\u0400-\u04FF]+', ' ', text)
    return text

In [77]:
def extract_text_from_html(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        soup = BeautifulSoup(file, 'html.parser')
        title_tag = soup.title
        title = clean(title_tag.get_text(strip=True)) if title_tag else ""

        news_detail_div = soup.find('div', class_='news-detail')
        text = clean(news_detail_div.get_text(strip=True)) if news_detail_div else ""
        return f"{title} {text}"

In [80]:
text_dir = 'data/page_texts'

for file_path in pages_list:
    text = extract_text_from_html(file_path)
    
    filename = os.path.basename(file_path)
    text_filename = os.path.splitext(filename)[0] + '.txt'
    text_file_path = os.path.join(text_dir, text_filename)
    
    with open(text_file_path, 'w', encoding='utf-8') as text_file:
        text_file.write(text)